In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import branca
import branca.colormap as cm
import os

import matplotlib as mpl
from shapely.geometry import Point
import contextily as cx
import geopandas
from geopandas import GeoDataFrame

#import plotly_express as px
import matplotlib.image as mpimg
import matplotlib.colors as mcolors

import random
import seaborn as sns

import math

from matplotlib.patches import Rectangle

from shapely.geometry import Polygon

#############   ENSURE YOUR RESULT DIRECTORY FACILITY FILE MATCHES   ###############

Directory = 'Results'
FacilitiesFile = 'Facilities.csv'  

def FacilityImportProcess(FacilitiesFile):
    Facs = pd.read_csv(FacilitiesFile)
    Facs

    Facs_base = Facs[['CITY', 'COUNTY', 'COUNTY_FIPS', 'END_USE', 'FACILITY_ID',
           'FINAL_NAICS_CODE', 'MECS_NAICS', 'REPORTING_YEAR', 'STATE', 
           'UNIT_NAME', 'UNIT_TYPE', 'Lat', 'Lon', 'LLType','Emissions 300','Emissions 550','Emissions 850',
           'geometry', 'FacilityName', 'Industry', 'Company']]

    #  'Temp_degC', 'Thermal MWh/hr'

    Facs['3-5'] = True
    Facs['5-8'] = True
    Facs.loc[Facs['Temp 300']-Facs['Temp 550'] == 0, '3-5'] = False
    Facs.loc[Facs['Temp 550']-Facs['Temp 850'] == 0, '5-8'] = False
    Facs

    Facs_300 = Facs_base.copy()
    Facs_550 = Facs_base.copy()
    Facs_850 = Facs_base.copy()

    Facs_300['Thermal MWh/hr'] = Facs['Demand 300']*1.1*277.778/8760
    Facs_300['Emissions'] = Facs['Emissions 300']
    Facs_300['Temp_degC'] = Facs['Temp 300']
    Facs_300['Temp_Req'] = 300
    Facs_300['Indexer'] = range(len(Facs_300.index))

    Facs_550['Thermal MWh/hr'] = Facs['Demand 550']*1.1*277.778/8760
    Facs_550['Emissions'] = Facs['Emissions 550']
    Facs_550['Temp_degC'] = Facs['Temp 550']
    Facs_550['Temp_Req'] = 550
    Facs_550['Indexer'] = range(len(Facs_550.index))
    Facs_550 = Facs_550.loc[Facs['3-5']]

    Facs_850['Thermal MWh/hr'] = Facs['Demand 850']*1.1*277.778/8760
    Facs_850['Emissions'] = Facs['Emissions 850']
    Facs_850['Temp_degC'] = Facs['Temp 850']
    Facs_850['Temp_Req'] = 850
    Facs_850['Indexer'] = range(len(Facs_850.index))
    Facs_850 = Facs_850.loc[Facs['5-8']]

    Facs_total = pd.concat((Facs_300,Facs_550,Facs_850)).reset_index(drop = True)

    Facs_total['FullIndex'] = range(len(Facs_total.index))

    F_IDS = Facs.FACILITY_ID.unique().tolist()
    Inds = []
    for f in F_IDS:
        Inds.append(Facs_total.loc[Facs_total['FACILITY_ID']==f,'FullIndex'].tolist())
    Fac_dict = {F_IDS[x]:Inds[x] for x in range(len(F_IDS))}

    return Facs_total, Fac_dict
Facs_total, Fac_dict = FacilityImportProcess(FacilitiesFile)


os.chdir(Directory)
os.listdir()

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

B_pd = pd.DataFrame()

for d in os.listdir():
    if '.' not in d:
        fs = os.listdir(d)
        for f in fs:
            if 'ALL' in f:
                print(d)
                res = pd.read_csv(d+'/'+f)
                res_B = res.loc[res.Type == 'B']
                res_B['Scenario'] = d
                
                B_pd = pd.concat((B_pd,res_B.loc[res_B.Year <=20]))
B_pd = B_pd.drop_duplicates(subset = ['Generator','Iteration','Year','Scenario', 'Facility'])
B_pd['Built'] = B_pd.groupby(['Generator','Facility','Scenario'])['Value'].transform('sum')

Scens = B_pd.Scenario.tolist()
B_pd['Scen_type'] = [x.split('_')[1] for x in B_pd['Scenario'].tolist()]
B_pd['NGP'] = [x.split('_')[0] for x in B_pd['Scenario'].tolist()]
B_pd['Scen_code'] = B_pd.Scen_type+'_'+B_pd.NGP
B_pd

In [ ]:
# Presents quick overview of total per-design deployment
pd.pivot_table(B_pd, values = 'Value',index = ['Scenario'], columns = ['Generator'], aggfunc=np.sum)

In [ ]:
# changes the numerical index of the generators to the SMR design and outputs a csv of these results

B_pd.Generator.replace('HTGR',0, inplace = True)
B_pd.Generator.replace('Microreactor',1, inplace = True)
B_pd.Generator.replace('PBR-HTGR',2, inplace = True)
B_pd.Generator.replace('iPWR',3, inplace = True)
B_piv = pd.pivot_table(B_pd, values = 'Value',index = ['NGP','Scen_type'], columns = ['Generator'], aggfunc=np.sum)
B_piv.to_csv('B_piv_table.csv')

In [ ]:
# Prepping the data for mapping and industry analysis

B_pd['OP_Max'] = B_pd.groupby(['Facility','Scenario','Generator'])['Value'].transform('sum')
B_pd = B_pd.loc[B_pd.OP_Max >0]
B_pd_slim = B_pd.drop_duplicates(subset=['Facility','Scenario','Generator'])
OP_f = B_pd_slim.Facility.tolist()
OP_map = pd.DataFrame()
OP_keys = []

for i in OP_f:
    OP_keys.append(list(Fac_dict.keys())[i])
    OP_map = pd.concat((OP_map,Facs_total.loc[Facs_total.FACILITY_ID == list(Fac_dict.keys())[i]].iloc[0]), axis = 1)
OP_map = OP_map.T
OP_Built = B_pd_slim
OP_Built.reset_index(drop = True, inplace = True)
OP_map.reset_index(drop = True, inplace = True)
for c in OP_Built.columns.tolist():
    print(c)
    if c != 'Unnamed: 0':
        OP_map[c] = OP_Built[c]
        
OP_map_g = geopandas.GeoDataFrame(OP_map, geometry=geopandas.points_from_xy(OP_map.Lon, OP_map.Lat))
OP_map_g.crs = {"init": "epsg:4326"}
OP_map_g = OP_map_g.to_crs(epsg=3857)

OP_map_g['NP_cap'] = OP_map_g.OP_Max
OP_map_g.loc[OP_map_g.Generator =='HTGR','NP_cap'] *=350
OP_map_g.loc[OP_map_g.Generator =='PBR-HTGR','NP_cap'] *=200
OP_map_g.loc[OP_map_g.Generator =='Microreactor','NP_cap'] *=20
OP_map_g.loc[OP_map_g.Generator =='iPWR','NP_cap'] *=1500
OP_map_g['Excess'] = OP_map_g.NP_cap>OP_map_g['Thermal MWh/hr']
OP_map_g['Remaining_NG'] = (OP_map_g['Thermal MWh/hr']-OP_map_g.NP_cap)*~OP_map_g.Excess
OP_map_g['Reduced_Ems'] = OP_map_g.Emissions
OP_map_g.loc[~OP_map_g['Excess'],'Reduced_Ems'] = (OP_map_g.NP_cap/OP_map_g['Thermal MWh/hr'])*OP_map_g.Emissions

OP_map_g['CF'] = 100
OP_map_g.loc[OP_map_g['Excess'],'CF'] = (OP_map_g['Thermal MWh/hr']/OP_map_g.NP_cap)*100

OP_map_g['Em_Rem'] = OP_map_g.groupby(['Scen_type', 'NGP','Scen_code'])['Emissions'].transform('sum')
OP_map_g['Em_Used'] = OP_map_g.groupby(['Scen_type', 'NGP','Scen_code'])['Emissions 850'].transform('sum')
OP_map_g['Reduced_Ems_sum'] = OP_map_g.groupby(['Scen_type', 'NGP','Scen_code'])['Reduced_Ems'].transform('sum')
OP_map_g